In [ ]:
from diffusers import StableDiffusionXLImg2ImgPipeline
import torch
from accelerate import infer_auto_device_map

# Carica il modello dalla piattaforma Hugging Face
model_id = "stabilityai/stable-diffusion-xl-base-1.0"

# Configura la pipeline
pipeline = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    model_id, 
    torch_dtype=torch.float32, 
    use_safetensors=True   # Per performance migliori
).to("cuda")

In [ ]:
from PIL import Image
import os
import matplotlib.pyplot as plt

FOLDER_PATH = "/home/samuele.cappelletti/Dataset/TrueFake/PreSocial/Real/FORLAB"
OUTPUT_FOLDER_PATH = "/home/samuele.cappelletti/Dataset_New/float32/TrueFake/PreSocial/Real/FORLAB_str_10"
FILE_FORMAT = ".png"

# Recupera lista img da FOLDER_PATH
images = [os.path.join(FOLDER_PATH, file) for file in os.listdir(FOLDER_PATH) if file.lower().endswith(FILE_FORMAT)]

os.makedirs(OUTPUT_FOLDER_PATH, exist_ok=True)  # Crea la directory se non esiste

print(os.path.exists(FOLDER_PATH))
print(os.path.exists(OUTPUT_FOLDER_PATH))
    

In [ ]:
# Prompt di esempio
prompt = " "
#negative = "Blurry details, low-quality rendering, distorted proportions, overly dark colors, excessive noise, outdated architecture, dull lighting, cartoonish appearance"
negative = " "

strength = 0.10
num_inference_steps=50
guidance_scale=10

height = 1024  # Altezza in pixel
width = 1024  # Larghezza in pixel
        
for image_path in images:
    
    input_image = Image.open(image_path)
    file_name_no_ext, ext = os.path.splitext(os.path.basename(input_image.filename))
    
    input_image = input_image.convert("RGB")
    input_image = input_image.resize((1024, 1024))
    
    # Generazione dell'immagine
    image = pipeline(
        prompt=prompt,
        image=input_image,
        strength=strength,
        num_inference_steps=num_inference_steps,    # Passi di inferenza (maggiore = più dettagli)
        guidance_scale=guidance_scale              # Peso per il prompt (maggiore = risultati più aderenti)
    ).images[0]
    
    # Salva l'immagine generata
    image.save(OUTPUT_FOLDER_PATH + "/" + file_name_no_ext + FILE_FORMAT)